In [0]:
%pip install -r requirements.txt
%restart_python

In [0]:
%load_ext autoreload
%autoreload 2

from lakebase_utils import LakebaseConnection
import pandas as pd

In [0]:
dbutils.widgets.text("user", "lars.liahagen@databricks.com")
username = dbutils.widgets.get("user")
dbutils.widgets.text("lakebase_instance_name", "smart-stock-db")
lakebase_instance_name = dbutils.widgets.get("lakebase_instance_name")
dbutils.widgets.text("catalog", "smart_stock")
catalog = dbutils.widgets.get("catalog")
dbutils.widgets.text("schema_forecast", "forecast")
schema_forecast = dbutils.widgets.get("schema_forecast")

conn = LakebaseConnection(username, lakebase_instance_name)

In [0]:
recommendations = spark.table(f"{catalog}.{schema_forecast}.inventory_forecast")
df_rec = recommendations.toPandas()

In [0]:
table_name = "inventory_forecast"
cols = [
    "product_id",
    "warehouse_id",
    "current_stock",
    "forecast_30_days",
    "reorder_point",
    "reorder_quantity",
    "confidence_score"
]

values_sql = []
for row in df_rec.itertuples():
    vals = []
    for col in cols:
        val = getattr(row, col)
        if pd.isnull(val):
            vals.append('NULL')
        elif isinstance(val, str):
            vals.append(f"'{val.replace("'", "''")}'")
        elif isinstance(val, int):
            vals.append(f"{val}")
        elif isinstance(val, float):
            vals.append(f"{val:.2f}")
        else:
            vals.append(str(int(val)))
    values_sql.append(f"({', '.join(vals)})")

insert_sql = f"""
INSERT INTO {table_name} ({', '.join(cols)})
VALUES\n  {',\n  '.join(values_sql)}
ON CONFLICT (product_id, warehouse_id)
DO UPDATE SET
  current_stock = EXCLUDED.current_stock,
  forecast_30_days = EXCLUDED.forecast_30_days,
  reorder_point = EXCLUDED.reorder_point,
  reorder_quantity = EXCLUDED.reorder_quantity,
  confidence_score = EXCLUDED.confidence_score;
"""

print(insert_sql)

In [0]:
conn.execute_statement(insert_sql)
print("Recommendations successfully written back to Postgres!")